In [2]:
import numpy as np
import pandas
import pm4py
from matplotlib import pyplot as plt
from sklearn.mixture import GaussianMixture
import scipy.stats as stats
import ot
import os
from tqdm import tqdm
import collections
import matplotlib.dates as md
import importlib
import pickle
import random
import CRPS.CRPS as pscore


pandas.set_option('display.max_columns', None)
#pandas.set_option('display.max_rows', None)


import sys
sys.path.append('../../TaskExecutionTimeMining/')
from event_log_transformer import *

#sys.path.append('../../Evaluation')
sys.path.append('../../Evaluation/')
from normal_evaluation.dumas_evaluation import *

In [3]:
import sys
sys.path.append('../../../src/TaskExecutionTimeMining/pix-framework-main/src')

with open('./dumas_model.pickle', 'rb') as f:
    dumas_model = pickle.load(f)

with open('./dumas_model_no_resource.pickle', 'rb') as f:
    dumas_model_no_resource = pickle.load(f)

In [4]:
import conduct_evaluation
get_pscores = lambda likelihoods : [pscore(likelihoods[1][i], likelihoods[2][k][3]).compute()[0] for i, k in enumerate(list(likelihoods[0].keys()))]
n_processes = 12

N = 1000

In [5]:
with open('../../../data/BPIC2017/transformed_event_logs//BPIC2017_train.pickle', 'rb') as f:
    train_data = pickle.load(f)

with open('../../../data/BPIC2017/transformed_event_logs/BPIC2017_test.pickle', 'rb') as f:
    test_data = pickle.load(f)

In [6]:
evaluator_train = conduct_evaluation.ConductEvaluation(dumas_model, SampleOutcomes_Dumas_Normal,
                                                 {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource_start',
                                                 },
                                     train_data, n=N, n_processes = n_processes)

evaluator_test = conduct_evaluation.ConductEvaluation(dumas_model, SampleOutcomes_Dumas_Normal,
                                                 {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource_start',
                                                 },
                                     test_data, n=N, n_processes = n_processes)

likelihoods_train = evaluator_train.sample_cases(plot_cases=False, multiprocessing=True)
likelihoods_test = evaluator_test.sample_cases(plot_cases=False, multiprocessing=True)

results = {
    "ll_train": np.mean([v.ln() for v in likelihoods_train[0].values()]),
    "ll_test": np.mean([v.ln() for v in likelihoods_test[0].values()]),
    "crsp_train": np.mean(get_pscores(likelihoods_train)),
    "crsp_test": np.mean(get_pscores(likelihoods_test))
}
results

100%|██████████| 6068/6068 [00:56<00:00, 108.13it/s]


{'ll_train': Decimal('-4.361289495047149513101817860'),
 'll_test': Decimal('-4.145566228697533014621765417'),
 'crsp_train': np.float64(718316.4196758654),
 'crsp_test': np.float64(718007.1490281759)}

In [7]:
evaluator_train = conduct_evaluation.ConductEvaluation(dumas_model_no_resource, SampleOutcomes_Dumas_Normal,
                                                 {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource_start',
                                                 },
                                     train_data, n=N, n_processes = n_processes)

evaluator_test = conduct_evaluation.ConductEvaluation(dumas_model_no_resource, SampleOutcomes_Dumas_Normal,
                                                 {
                                                        'activity_key' : 'concept:name',
                                                        'resource_key' : 'org:resource_start',
                                                 },
                                     test_data, n=N, n_processes = n_processes)

likelihoods_train = evaluator_train.sample_cases(plot_cases=False, multiprocessing=True)
likelihoods_test = evaluator_test.sample_cases(plot_cases=False, multiprocessing=True)

results = {
    "ll_train": np.mean([v.ln() for v in likelihoods_train[0].values()]),
    "ll_test": np.mean([v.ln() for v in likelihoods_test[0].values()]),
    "crsp_train": np.mean(get_pscores(likelihoods_train)),
    "crsp_test": np.mean(get_pscores(likelihoods_test))
}
results

100%|██████████| 6068/6068 [01:11<00:00, 84.65it/s] 


{'ll_train': Decimal('-4.328937963741169498158402575'),
 'll_test': Decimal('-4.070875191457320676632662063'),
 'crsp_train': np.float64(709320.1683225778),
 'crsp_test': np.float64(711259.819245299)}